In [ ]:
from pycdft import *
from ase.io import read

In [ ]:
ase_cell = read("./Zn2.cif")

In [ ]:
sample = Sample(ase_cell=ase_cell, n1=126, n2=126, n3=154, vspin=1)

In [ ]:
qboxdriver = QboxDriver(
    sample=sample,
    init_cmd="load gs.xml\nset xc PBE\nset wf_dyn JD\nset scf_tol 1.0E-8\n",
    scf_cmd="run 0 50 5"
)

In [ ]:
solver1 = CDFTSolver(job="scf", optimizer="secant", sample=sample, dft_driver=qboxdriver)
solver2 = solver1.copy()

In [ ]:
# add constraint to two solvers
ChargeTransferConstraint(
    sample=solver1.sample,
    donor=Fragment(solver1.sample, solver1.sample.atoms[0:1]),
    acceptor=Fragment(solver1.sample, solver1.sample.atoms[1:2]),
    V_init=-0.176,
    N0=1,
    N_tol=5E-5
)
ChargeTransferConstraint(
    sample=solver2.sample, 
    donor=Fragment(solver2.sample, solver2.sample.atoms[0:1]),
    acceptor=Fragment(solver2.sample, solver2.sample.atoms[1:2]),
    V_init=0.176,
    N0=-1, 
    N_tol=5E-5
)

In [ ]:
solver1.solve()

In [ ]:
solver2.solve()

In [ ]:
compute_elcoupling(solver1, solver2)